**GAN**

In [1]:
!git clone https://github.com/adcollin/AMPLify-Feedback.git
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

Cloning into 'AMPLify-Feedback'...
remote: Enumerating objects: 361, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 361 (delta 1), reused 7 (delta 1), pack-reused 351
Receiving objects: 100% (361/361), 65.84 MiB | 11.99 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Updating files: 100% (71/71), done.


In [2]:
seq_train = np.load("AMPLify-Feedback/processed_data/test/seq_cv.npy")
state_train = np.load("AMPLify-Feedback/processed_data/test/state_cv.npy")
label_train = np.load("AMPLify-Feedback/processed_data/test/label_cv.npy")
seq_val = np.load("AMPLify-Feedback/processed_data/test/seq_test.npy")
state_val = np.load("AMPLify-Feedback/processed_data/test/state_test.npy")
label_val = np.load("AMPLify-Feedback/processed_data/test/label_test.npy")
seq_train = tf.one_hot(seq_train.squeeze(), depth=43)
seq_val = tf.one_hot(seq_val.squeeze(), depth=43)

In [3]:
# Generator
def build_generator(seq_length, depth, latent_dim):
    inputs0 = layers.Input(shape=(latent_dim,), name="Input0")
    inputs1 = layers.Input(shape=(326,), name="Input1")
    x1 = layers.Dense(latent_dim, activation='tanh', name="Input1Transform")(inputs1)
    x = layers.Concatenate(name="Concat")([inputs0, x1])
    x = layers.Dense(256, input_dim=latent_dim, activation='relu', name="Dense0")(x)
    x = layers.BatchNormalization(name="BatchNorm0")(x)
    x = layers.Dense(seq_length*10, activation='tanh', name="DenseResize")(x)
    x = layers.Reshape((seq_length, 10), name="Reshape")(x)
    x = layers.GRU(256, return_sequences=True, name="GRU0")(x)
    x = layers.Dense(depth, activation="softmax", name="Output")(x)
    model = tf.keras.models.Model(inputs=[inputs0, inputs1], outputs=x)
    return model

In [4]:
# Discriminator
def build_discriminator(seq_length, depth):
    model = tf.keras.Sequential(name="discriminator")
    model.add(layers.Flatten(name="Flatten"))
    model.add(layers.Dense(512, activation='relu', name="Dense0"))
    model.add(layers.Dropout(0.3, name="Dropout"))
    model.add(layers.Dense(256, activation='relu', name="Dense1"))
    model.add(layers.Dense(1, activation='sigmoid', name="Output"))
    return model

In [5]:
# GAN
def compile_gan(generator, discriminator):
    discriminator.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    discriminator.trainable = False
    gan_input0 = layers.Input(shape=(latent_dim,))
    gan_input1 = layers.Input(shape=(326,))
    gan_output = discriminator(generator([gan_input0, gan_input1]))
    gan = tf.keras.Model([gan_input0, gan_input1], gan_output)
    gan.compile(loss='binary_crossentropy', optimizer='adam')
    return gan

In [6]:
def train_gan(generator, discriminator, gan, seq_train, state_train, epochs, batch_size, latent_dim):
    for epoch in range(epochs):
        for i in range(0, seq_train.shape[0], batch_size):
            real_sequences = seq_train[i:i + batch_size]
            state_train_batch = state_train[i:i + batch_size]
            current_batch_size = real_sequences.shape[0]

            # Generate Fake sequence
            noise = np.random.normal(0, 1, (current_batch_size, latent_dim))
            generated_sequences = generator.predict([noise, state_train_batch])

            # Labels for real and fake data
            real_labels = np.ones((current_batch_size, 1))
            fake_labels = np.zeros((current_batch_size, 1))

            # Train discriminator
            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch(real_sequences, real_labels)
            d_loss_fake = discriminator.train_on_batch(generated_sequences, fake_labels)
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
            discriminator.trainable = False

            # Train generator
            noise = np.random.normal(0, 1, (current_batch_size, latent_dim))
            g_loss = gan.train_on_batch([noise, state_train_batch], np.ones((current_batch_size, 1)))

            # Print the progress
            print(f"Epoch {epoch+1}/{epochs}, Batch {i//batch_size+1}, Discriminator Loss: {d_loss}, Generator Loss: {g_loss}")

In [7]:
latent_dim = 10  # Example latent dimension
seq_length = seq_train.shape[1]
depth = 43

generator = build_generator(seq_length, depth, latent_dim)
discriminator = build_discriminator(seq_length,depth)
gan = compile_gan(generator, discriminator)

# Train GAN
train_gan(generator, discriminator, gan, seq_train, state_train, epochs=5, batch_size=32, latent_dim=latent_dim)


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 18ms/step
Epoch 2/5, Batch 573, Discriminator Loss: [0.50040093 0.8125    ], Generator Loss: 1.4269317388534546
1/1 [==============================] - 0s 20ms/step
Epoch 2/5, Batch 574, Discriminator Loss: [0.61080068 0.75      ], Generator Loss: 1.1564409732818604
1/1 [==============================] - 0s 18ms/step
Epoch 2/5, Batch 575, Discriminator Loss: [0.82836203 0.53125   ], Generator Loss: 1.1378535032272339
1/1 [==============================] - 0s 18ms/step
Epoch 2/5, Batch 576, Discriminator Loss: [0.78342175 0.5       ], Generator Loss: 1.0513567924499512
1/1 [==============================] - 0s 20ms/step
Epoch 2/5, Batch 577, Discriminator Loss: [0.61953437 0.625     ], Generator Loss: 0.9229744672775269
1/1 [==============================] - 0s 18ms/step
Epoch 2/5, Batch 578, Discriminator Loss: [0.5735139 0.75     ], Generator Loss: 1.0731412172317505
1/1 [======================

In [8]:
# After GAN is trained

def generate_sequences(generator, latent_dim, num_sequences):
    noise = np.random.normal(0, 1, (num_sequences, latent_dim))
    bacteria = []
    for i in range(num_sequences):
      bacterium = np.zeros(326)
      bacterium[np.random.randint(0, 326)]=1
      bacteria.append([bacterium])
    bacteria = np.concatenate(bacteria, axis=0)
    generated_sequences = generator.predict([noise, bacteria])
    return generated_sequences


In [9]:
generated_seqs = generate_sequences(generator, latent_dim, num_sequences=10)

1/1 [==============================] - 0s 37ms/step


In [10]:
generated_seqs_argmax = tf.math.argmax(generated_seqs, axis=2)

In [11]:
generated_seqs_argmax[9]

<tf.Tensor: shape=(190,), dtype=int64, numpy=
array([ 3,  2, 18, 18, 18,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0])>

In [12]:
generator.save("AMPLify-Feedback/model_weights/PeptideGenerator.keras")
discriminator.save("AMPLify-Feedback/model_weights/PeptideDiscriminator.keras")

In [13]:
# np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_10",generated_seqs)

In [14]:
#Convert to One_hot
# generated_seqs_one_hot = tf.one_hot(generated_seqs.squeeze(), depth=43)
# np.save("/content/AMPLify-Feedback/processed_data/GAN_seq/generated_seqs_one_hot_10",generated_seqs_one_hot)